In [7]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from lore_sa.util import record2str, neuclidean

In [24]:
from lore_sa.dataset import Dataset

In [25]:
df = Dataset.from_csv('test/resources/adult.csv')
df.df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [26]:
df.df.drop(['fnlwgt', 'education-num'], inplace=True, axis=1)

In [60]:
from lore_sa.encoder_decoder import OneHotEnc
encoder = OneHotEnc()

In [61]:
df_enc = encoder.encode(df,'class')

categorical features idx  [1, 2, 3, 4, 5, 6, 7, 11]
[[39 'State-gov' 'Bachelors' ... 0 40 'United-States']
 [50 'Self-emp-not-inc' 'Bachelors' ... 0 13 'United-States']
 [38 'Private' 'HS-grad' ... 0 40 'United-States']
 ...
 [58 'Private' 'HS-grad' ... 0 40 'United-States']
 [22 'Private' 'HS-grad' ... 0 20 'United-States']
 [52 'Self-emp-inc' 'HS-grad' ... 0 40 'United-States']]


TypeError: '<' not supported between instances of 'float' and 'str'

In [51]:
a = ['State-gov' 'Self-emp-not-inc' 'Private' 'Private' 'Private' 'Self-emp-inc']
np.unique(a)

array(['State-govSelf-emp-not-incPrivatePrivatePrivateSelf-emp-inc'],
      dtype='<U58')

In [2]:
df, class_name = prepare_adult_dataset('adult.csv')
df, feature_names, class_values, numeric_columns, rdf, real_feature_names, features_map = prepare_dataset(
    df, class_name, encdec='onehot')

print(df.shape)
df.head()


(32561, 104)


,age,capital-gain,capital-loss,hours-per-week,workclass=Federal-gov,workclass=Local-gov,workclass=Never-worked,workclass=Private,workclass=Self-emp-inc,workclass=Self-emp-not-inc,...,native-country=Puerto-Rico,native-country=Scotland,native-country=South,native-country=Taiwan,native-country=Thailand,native-country=Trinadad&Tobago,native-country=United-States,native-country=Vietnam,native-country=Yugoslavia,class
0,39,2174,0,40,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,50,0,0,13,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,38,0,0,40,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,53,0,0,40,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,28,0,0,40,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df, feature_names, class_values, numeric_columns, rdf, real_feature_names, features_map = prepare_dataset(
    df, class_name, encdec='target')

feature_names

array(['age', 'capital-gain', 'capital-loss', 'hours-per-week',
       'workclass=Federal-gov', 'workclass=Local-gov',
       'workclass=Never-worked', 'workclass=Private',
       'workclass=Self-emp-inc', 'workclass=Self-emp-not-inc',
       'workclass=State-gov', 'workclass=Without-pay', 'education=10th',
       'education=11th', 'education=12th', 'education=1st-4th',
       'education=5th-6th', 'education=7th-8th', 'education=9th',
       'education=Assoc-acdm', 'education=Assoc-voc',
       'education=Bachelors', 'education=Doctorate', 'education=HS-grad',
       'education=Masters', 'education=Preschool',
       'education=Prof-school', 'education=Some-college',
       'marital-status=Divorced', 'marital-status=Married-AF-spouse',
       'marital-status=Married-civ-spouse',
       'marital-status=Married-spouse-absent',
       'marital-status=Never-married', 'marital-status=Separated',
       'marital-status=Widowed', 'occupation=Adm-clerical',
       'occupation=Armed-Forces', 'o

In [4]:
features_map

{0: {'age': 0},
 1: {'capital-gain': 1},
 2: {'capital-loss': 2},
 3: {'hours-per-week': 3},
 4: {'workclass=Federal-gov': 4},
 5: {'workclass=Local-gov': 5},
 6: {'workclass=Never-worked': 6},
 7: {'workclass=Private': 7},
 8: {'workclass=Self-emp-inc': 8},
 9: {'workclass=Self-emp-not-inc': 9},
 10: {'workclass=State-gov': 10},
 11: {'workclass=Without-pay': 11},
 12: {'education=10th': 12},
 13: {'education=11th': 13},
 14: {'education=12th': 14},
 15: {'education=1st-4th': 15},
 16: {'education=5th-6th': 16},
 17: {'education=7th-8th': 17},
 18: {'education=9th': 18},
 19: {'education=Assoc-acdm': 19},
 20: {'education=Assoc-voc': 20},
 21: {'education=Bachelors': 21},
 22: {'education=Doctorate': 22},
 23: {'education=HS-grad': 23},
 24: {'education=Masters': 24},
 25: {'education=Preschool': 25},
 26: {'education=Prof-school': 26},
 27: {'education=Some-college': 27},
 28: {'marital-status=Divorced': 28},
 29: {'marital-status=Married-AF-spouse': 29},
 30: {'marital-status=Marrie

In [5]:
test_size = 0.30
random_state = 0

X_train, X_test, Y_train, Y_test = train_test_split(df[feature_names].values, df[class_name].values,
                                                    test_size=test_size,
                                                    random_state=random_state,
                                                    stratify=df[class_name].values)

_, K, _, _ = train_test_split(rdf[real_feature_names].values, rdf[class_name].values,
                              test_size=test_size,
                              random_state=random_state,
                              stratify=df[class_name].values)


# Train a random forest model

bb = RandomForestClassifier(n_estimators=20, random_state=random_state)
# bb = MLPClassifier(random_state=random_state)
bb.fit(X_train, Y_train)

# encapulating predition methods in functions
def bb_predict(X):
    return bb.predict(X)

def bb_predict_proba(X):
    return bb.predict_proba(X)


# example of predictions
Y_pred = bb_predict(X_test)

print('Accuracy %.3f' % accuracy_score(Y_test, Y_pred))
print('F1-measure %.3f' % f1_score(Y_test, Y_pred))

Accuracy 0.843
F1-measure 0.648


In [6]:
from lore_sa.neighgen import RandomGenerator

gen = RandomGenerator(bb_predict,feature_names,feature_names,len(feature_names),len(real_feature_names),numeric_columns)

In [7]:
from importlib import reload
import lore_sa
from lore_sa.lorem import LOREM


from lore_sa.neighgen import RandomGenerator

i2e = 3
x = X_test[i2e]
gen.feature_values = X_test#[i2e]
print('x = %s' % record2str(x, feature_names, numeric_columns))
print('')

bb_outcome = bb_predict(x.reshape(1, -1))[0]
bb_outcome_str = class_values[bb_outcome]

print('bb(x) = { %s }' % bb_outcome_str)
print('')



lore_explainer = LOREM(X_test, bb_predict, bb_predict_proba, feature_names, class_name, class_values, numeric_columns, features_map,
                       neigh_gen = gen, random_state=random_state, ngen=10, verbose=True, encdec=None, K_transformed=X_test)

exp = lore_explainer.explain_instance_stable(x, samples=1000, use_weights=True, metric=neuclidean)

print(exp)

x = { age = 34, capital-gain = 0, capital-loss = 0, hours-per-week = 40, workclass=Federal-gov = 0, workclass=Local-gov = 0, workclass=Never-worked = 0, workclass=Private = 1, workclass=Self-emp-inc = 0, workclass=Self-emp-not-inc = 0, workclass=State-gov = 0, workclass=Without-pay = 0, education=10th = 0, education=11th = 0, education=12th = 0, education=1st-4th = 0, education=5th-6th = 0, education=7th-8th = 0, education=9th = 0, education=Assoc-acdm = 0, education=Assoc-voc = 0, education=Bachelors = 1, education=Doctorate = 0, education=HS-grad = 0, education=Masters = 0, education=Preschool = 0, education=Prof-school = 0, education=Some-college = 0, marital-status=Divorced = 0, marital-status=Married-AF-spouse = 0, marital-status=Married-civ-spouse = 1, marital-status=Married-spouse-absent = 0, marital-status=Never-married = 0, marital-status=Separated = 0, marital-status=Widowed = 0, occupation=Adm-clerical = 0, occupation=Armed-Forces = 0, occupation=Craft-repair = 0, occupation

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished


In [10]:
help(exp)

Help on Explanation in module lore_sa.explanation object:

class Explanation(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  cstr(self)
 |  
 |  rstr(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

